# ***God Father***

In [ ]:
# God Father
! gdown --id 1PdrcSiFoJCydWkAWymWIBqmpgMHH_3qB
! gdown --id 1xs7R3SW_QTfnwzl2ByiBGEZ6F4_TaT5s

: 

In [ ]:
import librosa
import IPython.display as ipd

: 

In [ ]:
x, sr = librosa.load('/content/The Godfather Theme Song.mp3') 
ipd.Audio(x, rate=sr,autoplay=True)

: 

In [ ]:
ipd.Image('/content/wp9342478-the-godfather-part-ii-1974-wallpapers.jpg')

: 

# ***Downloading Content***

In [ ]:
%matplotlib inline

: 

In [ ]:
from __future__ import print_function, division
import pandas as pd 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

: 

In [ ]:
! gdown --id 1pL8D87UZ7ioj43ob14pPkCvJbQdTOwme

: 

In [ ]:
! gdown --id 1fbsnjPsrD2hBWSZdmDmwJpHqsVhQfvhm

: 

In [ ]:
!unzip "/content/sprints-ai-and-ml-competition-2022-round-2.zip" -d "/content/sprints-ai-and-ml-competition-2022-round-2"

: 

# ***Spliting Data into train and val***

In [ ]:
!pip install split-folders[full]
import splitfolders
# spliting Training to Train and val
split_dir = '/content/sprints-ai-and-ml-competition-2022-round-2/Scenes training set'

splitfolders.ratio(split_dir,output='output',ratio=(.85,.15))

: 

# ***Data Augmentation***

In [ ]:


data_transforms = {
     


    'train': transforms.Compose([
        transforms.Resize((150,150)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(),
        
        #transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'val': transforms.Compose([
        transforms.Resize((150,150)),
        transforms.CenterCrop(150),
    
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}


data_dir = '/content/output'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=24,
                                             shuffle=True, num_workers=14)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



: 

# ***Function to Train the Model***

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):   # we need to change number of epochs
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
from socket import socket    

: 

In [ ]:
# forcing using gpu
import tensorflow as tf
tf.test.gpu_device_name()

: 

# ***ResNet 50***

In [ ]:
model_ft50 = models.resnet50(pretrained=True)
num_ftrs = model_ft50.fc.in_features

model_ft50.fc = nn.Linear(num_ftrs, 6)

model_ft = model_ft50.to(device)

criterion = nn.CrossEntropyLoss()


optimizer_ft = optim.SGD(model_ft50.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs           
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

: 

In [ ]:
#model_ft50 = train_model(model_ft50, criterion, optimizer_ft, exp_lr_scheduler,
#                   num_epochs=15)

: 

: 

# ***ResNest 18***

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 6)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()


optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs           
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

: 

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                   num_epochs=15)

: 

# ***ConvNext_Tiny***

In [ ]:
model_conv = torchvision.models.convnext_tiny(pretrained=True)

model_conv.classifier[2]=nn.Linear(model_conv.classifier[2].in_features, 6)


model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()


optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs           
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

: 

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,exp_lr_scheduler, num_epochs=17)

: 

# ***Saving Model***

In [ ]:
save_dir='/content/Trained_model'
torch.save(model_conv,save_dir) #SAVES THE TRAINED MODEL


: 

# ***Loading Model***

In [ ]:
model=torch.load('/content/Trained_model')
model.eval()

: 

# ***Testing Phase***

In [ ]:
from re import X
import PIL.Image as im
#from PIL import Image
import os
from os import listdir
from glob import glob
def predict(model,image_transforms,images_paths,classes):
    list1=[]
    list2=[]
    for i in images_paths:
     print(i)
     model=model.eval()
     image = im.open(i)
     image=image_transforms(image).to('cuda').float()
     image=image.unsqueeze(0)
     output=model(image)
     _, predicted=torch.max(output.data,1)

     print(i[-9:],predicted.item())
     list1.append(i[-9:])
     list1.append(predicted.item())
     list2.append(list1)
     list1=[]
    return list2





: 

In [ ]:
Test_dir='/content/sprints-ai-and-ml-competition-2022-round-2/Scenes testing test'

tranform_test = transforms.Compose([transforms.Resize((150,150)),transforms.CenterCrop(150),
        transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
                 ])


: 

In [ ]:
x=os.listdir(Test_dir)

list_path=[]
for i in range(len(x)):
     list_path.append(Test_dir+"/"+str(x[i]))

: 

In [ ]:
data_frame=predict(model,tranform_test,list_path,class_names)

: 

# ***Importing to CSV***

In [ ]:
import pandas as pd
New_Data_Frame=pd.DataFrame(data_frame)
New_Data_Frame.to_csv('/content/sprints-ai-and-ml-competition-2022-round-2/submit.csv',index=False)


: 

## ***Draft Models***

# ***Resnet 18***

In [ ]:

model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 6)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

: 

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=80)

: 

# **ResNet 101**

In [ ]:
# using 1 model [resnet18] pretrained
model_ft = models.resnet101(pretrained=True)
num_ftrs = model_ft.fc.in_features
#print(model_ft)

model_ft.classifier = nn.Linear(num_ftrs, 6)  # changed to 6 Classes

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

: 

In [ ]:
#model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=60)

: 